🔴 Avancé | ⏱ 45 min | 🔑 Concepts : timeit, cProfile, memory_profiler, line_profiler

# 07 - Profiling et Optimisation de Performance

## Objectifs

- Comprendre quand et pourquoi profiler son code
- Mesurer le temps d'exécution avec **timeit**
- Profiler un programme complet avec **cProfile**
- Profiler ligne par ligne avec **line_profiler**
- Mesurer l'utilisation mémoire avec **memory_profiler**
- Optimiser le code Pandas et NumPy

## Prérequis

- Python 3.8+
- Connaissance de base de Python
- Familiarité avec Pandas (optionnel)

## 1. Pourquoi profiler ?

### Citation célèbre

> **"Premature optimization is the root of all evil."**  
> — Donald Knuth

### Quand profiler ?

✅ **Profiler quand** :
- Le code est trop lent pour les utilisateurs
- Vous avez des deadlines de performance à respecter
- La mémoire explose (out of memory)
- Vous voulez identifier les goulots d'étranglement

❌ **Ne PAS profiler** :
- Avant même d'avoir du code qui fonctionne
- Sans mesures concrètes ("je pense que c'est lent")
- Sans objectif de performance défini

### Règle d'or

1. **Make it work** (faites fonctionner le code)
2. **Make it right** (rendez-le correct)
3. **Make it fast** (optimisez si nécessaire)

### Loi de Pareto (80/20)

**80% du temps d'exécution** est souvent consommé par **20% du code**.

→ Trouvez ces 20% avec le profiling !

## 2. timeit : Mesurer le temps d'exécution

**timeit** est l'outil standard pour mesurer le temps d'exécution de petits morceaux de code.

In [ ]:
import timeit

# Mesurer une simple opération
time = timeit.timeit('x = 1 + 1', number=1_000_000)
print(f"Temps pour 1M additions: {time:.4f}s")

# Avec setup
time = timeit.timeit(
    'sum(data)',
    setup='data = list(range(1000))',
    number=10_000
)
print(f"Temps pour 10K sum(): {time:.4f}s")

### timeit dans Jupyter : %%timeit magic

In [ ]:
# Mesurer une seule ligne
%timeit sum(range(1000))

In [ ]:
# Mesurer une cellule entière
%%timeit
total = 0
for i in range(1000):
    total += i

### Comparaison : list vs generator

In [ ]:
# List comprehension
%timeit [x**2 for x in range(10000)]

In [ ]:
# Generator expression
%timeit list(x**2 for x in range(10000))

In [ ]:
# Generator sans matérialisation (création seulement)
%timeit (x**2 for x in range(10000))

### Comparaison : string concatenation

In [ ]:
# ❌ Concaténation naïve (lent)
%%timeit
result = ""
for i in range(1000):
    result += str(i)

In [ ]:
# ✅ join() (rapide)
%%timeit
result = "".join(str(i) for i in range(1000))

## 3. cProfile : Profiler un programme complet

**cProfile** analyse où le temps est dépensé dans tout le programme.

In [ ]:
%%writefile slow_program.py
"""Programme avec plusieurs fonctions à profiler."""
import time

def fonction_rapide():
    """Fonction rapide."""
    return sum(range(100))

def fonction_lente():
    """Fonction lente."""
    time.sleep(0.1)
    return sum(range(1000000))

def fonction_moyenne():
    """Fonction moyennement lente."""
    time.sleep(0.05)
    return sum(range(100000))

def main():
    """Point d'entrée."""
    for _ in range(5):
        fonction_rapide()
    
    for _ in range(3):
        fonction_lente()
    
    for _ in range(10):
        fonction_moyenne()

if __name__ == "__main__":
    main()

In [ ]:
# Profiler avec cProfile
!python -m cProfile -s cumulative slow_program.py

### Analyser les résultats avec pstats

In [ ]:
import cProfile
import pstats
from pstats import SortKey

# Profiler et sauvegarder
!python -m cProfile -o profile.stats slow_program.py

# Analyser
stats = pstats.Stats('profile.stats')
stats.strip_dirs()
stats.sort_stats(SortKey.CUMULATIVE)
stats.print_stats(10)  # Top 10 fonctions

### Colonnes cProfile

| Colonne | Description |
|---------|-------------|
| **ncalls** | Nombre d'appels |
| **tottime** | Temps total (sans sous-fonctions) |
| **percall** | tottime / ncalls |
| **cumtime** | Temps cumulatif (avec sous-fonctions) |
| **percall** | cumtime / ncalls |
| **filename:lineno(function)** | Fonction |

## 4. line_profiler : Profiler ligne par ligne

**line_profiler** permet de voir le temps passé sur chaque ligne de code.

In [ ]:
!pip install line_profiler -q

In [ ]:
%%writefile line_profile_example.py
"""Exemple pour line_profiler."""

@profile  # Décorateur magique pour kernprof
def traiter_donnees(n):
    """Traite des données."""
    # Étape 1 : Créer la liste
    data = list(range(n))
    
    # Étape 2 : Calculer les carrés
    carres = [x**2 for x in data]
    
    # Étape 3 : Filtrer les pairs
    pairs = [x for x in carres if x % 2 == 0]
    
    # Étape 4 : Somme
    total = sum(pairs)
    
    return total

if __name__ == "__main__":
    result = traiter_donnees(100000)

In [ ]:
# Profiler ligne par ligne
!kernprof -l -v line_profile_example.py

### Dans Jupyter avec %lprun

In [ ]:
%load_ext line_profiler

In [ ]:
def ma_fonction(n):
    data = list(range(n))
    carres = [x**2 for x in data]
    pairs = [x for x in carres if x % 2 == 0]
    return sum(pairs)

%lprun -f ma_fonction ma_fonction(10000)

## 5. memory_profiler : Mesurer l'utilisation mémoire

**memory_profiler** mesure la mémoire consommée ligne par ligne.

In [ ]:
!pip install memory_profiler -q

In [ ]:
%%writefile memory_example.py
"""Exemple memory_profiler."""
from memory_profiler import profile

@profile
def fonction_gourmande():
    """Fonction qui consomme de la mémoire."""
    # Liste de 10M entiers
    grande_liste = list(range(10_000_000))
    
    # Copie de la liste
    copie = grande_liste[:]
    
    # Transformation
    doubles = [x * 2 for x in grande_liste]
    
    # Nettoyage
    del grande_liste
    del copie
    
    return sum(doubles)

if __name__ == "__main__":
    result = fonction_gourmande()

In [ ]:
# Profiler la mémoire
!python memory_example.py

### Dans Jupyter avec %memit

In [ ]:
%load_ext memory_profiler

In [ ]:
# Mesurer la mémoire d'une ligne
%memit liste = list(range(1_000_000))

In [ ]:
# Comparer list vs generator
print("List:")
%memit liste = [x**2 for x in range(1_000_000)]

print("\nGenerator:")
%memit gen = (x**2 for x in range(1_000_000))

## 6. Optimisation Pandas

Pandas peut être très gourmand en mémoire. Voici comment l'optimiser.

In [ ]:
import pandas as pd
import numpy as np

# Créer un DataFrame de test
df = pd.DataFrame({
    'id': range(1_000_000),
    'category': np.random.choice(['A', 'B', 'C'], 1_000_000),
    'value': np.random.randn(1_000_000),
    'is_active': np.random.choice([True, False], 1_000_000)
})

print(f"Mémoire utilisée: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

### Optimisation 1 : Choisir les bons dtypes

In [ ]:
# Avant optimisation
print("Avant:")
print(df.dtypes)
print(f"Mémoire: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Optimisation des types
df_optimized = df.copy()
df_optimized['id'] = df_optimized['id'].astype('int32')  # int64 → int32
df_optimized['category'] = df_optimized['category'].astype('category')  # object → category
df_optimized['value'] = df_optimized['value'].astype('float32')  # float64 → float32

print("\nAprès:")
print(df_optimized.dtypes)
print(f"Mémoire: {df_optimized.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

reduction = (1 - df_optimized.memory_usage(deep=True).sum() / df.memory_usage(deep=True).sum()) * 100
print(f"\nRéduction: {reduction:.1f}%")

### Optimisation 2 : Éviter les copies

In [ ]:
# ❌ LENT : Crée des copies
%timeit df[df['value'] > 0][['id', 'value']]

# ✅ RAPIDE : Pas de copie intermédiaire
%timeit df.loc[df['value'] > 0, ['id', 'value']]

### Optimisation 3 : Vectorisation vs boucles

In [ ]:
# Petit DataFrame pour le test
df_small = df.head(10000).copy()

# ❌ TRÈS LENT : Boucle Python
def avec_boucle(df):
    result = []
    for val in df['value']:
        result.append(val * 2)
    return result

%timeit avec_boucle(df_small)

# ✅ RAPIDE : Vectorisation
%timeit df_small['value'] * 2

## 7. Optimisation NumPy

In [ ]:
import numpy as np

# Liste vs NumPy array
liste = list(range(1_000_000))
array = np.arange(1_000_000)

print("Liste Python:")
%timeit [x * 2 for x in liste]

print("\nNumPy array:")
%timeit array * 2

In [ ]:
# Opérations matricielles
A = np.random.rand(1000, 1000)
B = np.random.rand(1000, 1000)

# NumPy optimisé (BLAS/LAPACK)
%timeit np.dot(A, B)

## Pièges courants

### 1. Micro-optimisation inutile

In [ ]:
# ❌ MAUVAIS : Optimiser du code qui n'est pas le goulot
# Gagner 0.001s sur une fonction appelée une fois ne sert à rien

# ✅ BON : Profiler d'abord, optimiser ensuite
# Concentrez-vous sur les 20% qui prennent 80% du temps

### 2. Profiler du code non représentatif

In [ ]:
# ❌ MAUVAIS : Profiler avec des données de test minuscules
# df = pd.DataFrame({'a': [1, 2, 3]})

# ✅ BON : Profiler avec des données réalistes
# df = pd.read_csv('production_data_10M_rows.csv')

### 3. Ignorer la mémoire

In [ ]:
# ❌ MAUVAIS : Optimiser seulement la vitesse
# Peut consommer 100GB de RAM !

# ✅ BON : Équilibrer vitesse et mémoire
# Utiliser des generators, chunking, etc.

## Mini-Exercices

### Exercice 1 : Comparer list vs generator

Comparez la vitesse et la mémoire de list comprehension vs generator expression pour créer les 10 millions premiers carrés.

In [ ]:
# Votre solution ici


### Exercice 2 : Optimiser du code Pandas

Optimisez ce code Pandas pour réduire l'utilisation mémoire de 50%+ :

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    'user_id': range(1_000_000),
    'country': np.random.choice(['FR', 'US', 'UK', 'DE'], 1_000_000),
    'age': np.random.randint(18, 80, 1_000_000),
    'score': np.random.randn(1_000_000),
    'is_premium': np.random.choice([True, False], 1_000_000)
})

print(f"Mémoire avant: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# TODO: Optimisez ce DataFrame


### Exercice 3 : Profiler et optimiser

Profilez cette fonction et optimisez-la :

In [ ]:
def fonction_lente(n):
    """Fonction à optimiser."""
    # Créer une liste de tous les nombres
    nombres = []
    for i in range(n):
        nombres.append(i)
    
    # Filtrer les pairs
    pairs = []
    for num in nombres:
        if num % 2 == 0:
            pairs.append(num)
    
    # Calculer les carrés
    carres = []
    for num in pairs:
        carres.append(num ** 2)
    
    # Somme
    total = 0
    for carre in carres:
        total += carre
    
    return total

# TODO: Profilez avec timeit, puis optimisez


## Solutions

### Solution Exercice 1

In [ ]:
# Comparaison vitesse
print("=== VITESSE ===")
print("List comprehension:")
%timeit liste = [x**2 for x in range(10_000_000)]

print("\nGenerator (création):")
%timeit gen = (x**2 for x in range(10_000_000))

print("\nGenerator (matérialisé):")
%timeit list(x**2 for x in range(10_000_000))

# Comparaison mémoire
print("\n=== MÉMOIRE ===")
print("List comprehension:")
%memit liste = [x**2 for x in range(10_000_000)]

print("\nGenerator (non matérialisé):")
%memit gen = (x**2 for x in range(10_000_000))

print("\n✅ Conclusion: Generator est instantané et consomme ~0 mémoire (avant matérialisation)")

### Solution Exercice 2

In [ ]:
import pandas as pd
import numpy as np

# DataFrame original
df = pd.DataFrame({
    'user_id': range(1_000_000),
    'country': np.random.choice(['FR', 'US', 'UK', 'DE'], 1_000_000),
    'age': np.random.randint(18, 80, 1_000_000),
    'score': np.random.randn(1_000_000),
    'is_premium': np.random.choice([True, False], 1_000_000)
})

avant = df.memory_usage(deep=True).sum() / 1024**2
print(f"Avant optimisation: {avant:.2f} MB")
print(df.dtypes)

# Optimisation
df_opt = df.copy()
df_opt['user_id'] = df_opt['user_id'].astype('int32')  # int64 → int32
df_opt['country'] = df_opt['country'].astype('category')  # object → category
df_opt['age'] = df_opt['age'].astype('int8')  # int64 → int8 (18-80 tient dans int8)
df_opt['score'] = df_opt['score'].astype('float32')  # float64 → float32
# is_premium est déjà bool (optimal)

apres = df_opt.memory_usage(deep=True).sum() / 1024**2
print(f"\nAprès optimisation: {apres:.2f} MB")
print(df_opt.dtypes)

reduction = (1 - apres / avant) * 100
print(f"\n✅ Réduction: {reduction:.1f}%")

### Solution Exercice 3

In [ ]:
# Version originale (lente)
def fonction_lente(n):
    nombres = []
    for i in range(n):
        nombres.append(i)
    
    pairs = []
    for num in nombres:
        if num % 2 == 0:
            pairs.append(num)
    
    carres = []
    for num in pairs:
        carres.append(num ** 2)
    
    total = 0
    for carre in carres:
        total += carre
    
    return total

print("Version originale:")
%timeit fonction_lente(100000)

# Version optimisée
def fonction_rapide(n):
    """Version optimisée avec comprehensions et sum()."""
    return sum((x ** 2 for x in range(n) if x % 2 == 0))

print("\nVersion optimisée:")
%timeit fonction_rapide(100000)

# Version NumPy (encore plus rapide)
def fonction_numpy(n):
    """Version NumPy."""
    arr = np.arange(n)
    pairs = arr[arr % 2 == 0]
    return (pairs ** 2).sum()

print("\nVersion NumPy:")
%timeit fonction_numpy(100000)

# Vérifier que les résultats sont identiques
assert fonction_lente(1000) == fonction_rapide(1000) == fonction_numpy(1000)
print("\n✅ Toutes les versions donnent le même résultat")